# Kubernetes操作

## Kubernetesリソースの作成


In [217]:
!date

Fri Feb 14 11:31:17 UTC 2020


In [218]:
import json

f = open('res_info.json', 'r')
res_info = json.load(f)
f.close()

res_info

{'vender': 'aws',
 'clusterName': 'aws-eks-200214',
 'nodeGroupName': 'ng-200214',
 'workerRolearn': 'eksctl-aws-eks-200214-nodegroup-n-NodeInstanceRole-L49KCDT5ANC5',
 'NodeInstanceRole': 'eksctl-aws-eks-200214-nodegroup-n-NodeInstanceRole-L49KCDT5ANC5',
 'FileSystemId': 'fs-83a42529',
 'region': 'us-west-2',
 'db_address': 'rds-topse.ciph33ijhh38.us-west-2.rds.amazonaws.com',
 'vpcID': 'vpc-07b429d6780e6ec5f'}

### Wordpressの準備

In [219]:
!cat yaml/wordpress-deploy.yaml

apiVersion: apps/v1

kind: Deployment
metadata:
  name: wordpress
  labels:
    app: wordpress
spec:
  replicas: 2
  selector:
    matchLabels:
      app: wordpress
  template:
    metadata:
      labels:
        app: wordpress
    spec:
      containers:
        - image: wordpress:4.8-apache
          name: wordpress
          volumeMounts:
            - name: efs-pvc
              mountPath: "/var/www/html/wp-content/uploads"
          env:
          - name: WORDPRESS_DB_HOST
            value: rds-topse.ciph33ijhh38.us-west-2.rds.amazonaws.com
          - name: WORDPRESS_DB_PASSWORD
            value: password

          ports:
            - containerPort: 80
              name: wordpress
          lifecycle:
            postStart:
              exec:
                command: ["/bin/sh", "-c", "chown -R www-data:www-data /var/www/html/wp-content/uploads"]
      volumes:
        - name: efs-pvc
          persistentVolumeClaim:
            claimName: fileserver-claim

In [220]:
import re

before_str=r'''- name: WORDPRESS_DB_HOST
            value: \S*'''

after_str='''- name: WORDPRESS_DB_HOST
            value: {db_address}
'''.format(db_address=res_info['db_address']).strip()

f = open('yaml/wordpress-deploy.yaml','r')
body = f.read()
new_body = re.sub(before_str, after_str, body, flags=re.DOTALL)
f.close()

f = open('yaml/wordpress-deploy.yaml','w')
f.write(new_body)
f.close()

In [221]:
!cat yaml/wordpress-deploy.yaml

apiVersion: apps/v1

kind: Deployment
metadata:
  name: wordpress
  labels:
    app: wordpress
spec:
  replicas: 2
  selector:
    matchLabels:
      app: wordpress
  template:
    metadata:
      labels:
        app: wordpress
    spec:
      containers:
        - image: wordpress:4.8-apache
          name: wordpress
          volumeMounts:
            - name: efs-pvc
              mountPath: "/var/www/html/wp-content/uploads"
          env:
          - name: WORDPRESS_DB_HOST
            value: rds-topse.ciph33ijhh38.us-west-2.rds.amazonaws.com
          - name: WORDPRESS_DB_PASSWORD
            value: password

          ports:
            - containerPort: 80
              name: wordpress
          lifecycle:
            postStart:
              exec:
                command: ["/bin/sh", "-c", "chown -R www-data:www-data /var/www/html/wp-content/uploads"]
      volumes:
        - name: efs-pvc
          persistentVolumeClaim:
            claimName: fileserver-claim

### EFSの準備
configmap.yaml ファイルで、file.system.id および aws.region を EFS に適した値に変更します。 


In [222]:
!cat yaml/configmap.yaml

apiVersion: v1
kind: ConfigMap
metadata:
  name: efs-provisioner
data:
  file.system.id: fs-83a42529
  aws.region: us-west-2
  provisioner.name: example.com/aws-efs
  dns.name: ""


In [223]:
import re

before_str=r'''file.system.id: \S*
  aws.region: \S*'''
after_str='''file.system.id: {FileSystemId}
  aws.region: {region}   
'''.format(FileSystemId=res_info['FileSystemId'],region=res_info['region']).strip()

f = open('yaml/configmap.yaml','r')
body = f.read()
new_body = re.sub(before_str, after_str, body, flags=re.DOTALL)
f.close()

f = open('yaml/configmap.yaml','w')
f.write(new_body)
f.close()


In [224]:
!cat yaml/configmap.yaml

apiVersion: v1
kind: ConfigMap
metadata:
  name: efs-provisioner
data:
  file.system.id: fs-83a42529
  aws.region: us-west-2
  provisioner.name: example.com/aws-efs
  dns.name: ""


In [225]:
!cat yaml/class.yaml

kind: StorageClass
apiVersion: storage.k8s.io/v1
metadata:
  name: aws-efs
provisioner: example.com/aws-efs


5.    deployment.yaml ファイルで、ボリューム設定を / の path に変更します。serverのドメインも変更します。

In [226]:
!cat yaml/deployment.yaml

apiVersion: v1
kind: ServiceAccount
metadata:
  name: efs-provisioner
---
kind: Deployment
apiVersion: apps/v1
metadata:
  name: efs-provisioner
spec:
  replicas: 1
  selector:
    matchLabels:
      app: efs-provisioner
  strategy:
    type: Recreate 
  template:
    metadata:
      labels:
        app: efs-provisioner
    spec:
      serviceAccount: efs-provisioner
      containers:
        - name: efs-provisioner
          image: quay.io/external_storage/efs-provisioner:latest
          env:
            - name: FILE_SYSTEM_ID
              valueFrom:
                configMapKeyRef:
                  name: efs-provisioner
                  key: file.system.id
            - name: AWS_REGION
              valueFrom:
                configMapKeyRef:
                  name: efs-provisioner
                  key: aws.region
            - name: DNS_NAME
              valueFrom:
                configMapKeyRef:
                  name: efs-provisioner
                  key: dns.name
       

In [227]:
import re

before_str=r'''server: \S*
            path: \S*'''

after_str='''server: {FileSystemId}.efs.{region}.amazonaws.com
            path: /
'''.format(FileSystemId=res_info['FileSystemId'],region=res_info['region']).strip()

f = open('yaml/deployment.yaml','r')
body = f.read()
new_body = re.sub(before_str, after_str, body, flags=re.DOTALL)
f.close()

f = open('yaml/deployment.yaml','w')
f.write(new_body)
f.close()

In [228]:
!cat yaml/deployment.yaml

apiVersion: v1
kind: ServiceAccount
metadata:
  name: efs-provisioner
---
kind: Deployment
apiVersion: apps/v1
metadata:
  name: efs-provisioner
spec:
  replicas: 1
  selector:
    matchLabels:
      app: efs-provisioner
  strategy:
    type: Recreate 
  template:
    metadata:
      labels:
        app: efs-provisioner
    spec:
      serviceAccount: efs-provisioner
      containers:
        - name: efs-provisioner
          image: quay.io/external_storage/efs-provisioner:latest
          env:
            - name: FILE_SYSTEM_ID
              valueFrom:
                configMapKeyRef:
                  name: efs-provisioner
                  key: file.system.id
            - name: AWS_REGION
              valueFrom:
                configMapKeyRef:
                  name: efs-provisioner
                  key: aws.region
            - name: DNS_NAME
              valueFrom:
                configMapKeyRef:
                  name: efs-provisioner
                  key: dns.name
       

### ALBの準備

In [229]:
!cat yaml/rbac-role.yaml

---
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRole
metadata:
  labels:
    app.kubernetes.io/name: alb-ingress-controller
  name: alb-ingress-controller
rules:
  - apiGroups:
      - ""
      - extensions
    resources:
      - configmaps
      - endpoints
      - events
      - ingresses
      - ingresses/status
      - services
    verbs:
      - create
      - get
      - list
      - update
      - watch
      - patch
  - apiGroups:
      - ""
      - extensions
    resources:
      - nodes
      - pods
      - secrets
      - services
      - namespaces
    verbs:
      - get
      - list
      - watch
---
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRoleBinding
metadata:
  labels:
    app.kubernetes.io/name: alb-ingress-controller
  name: alb-ingress-controller
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: alb-ingress-controller
subjects:
  - kind: ServiceAccount
    name: alb-ingress-controller
    namespace: kube-system
---
apiV

In [230]:
!wget -O yaml/alb-ingress-controller.yaml https://raw.githubusercontent.com/kubernetes-sigs/aws-alb-ingress-controller/v1.1.4/docs/examples/alb-ingress-controller.yaml

--2020-02-14 11:31:26--  https://raw.githubusercontent.com/kubernetes-sigs/aws-alb-ingress-controller/v1.1.4/docs/examples/alb-ingress-controller.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3129 (3.1K) [text/plain]
Saving to: ‘yaml/alb-ingress-controller.yaml’

yaml/alb-ingress-co 100%[===================>]   3.06K  --.-KB/s    in 0s      

2020-02-14 11:31:27 (37.8 MB/s) - ‘yaml/alb-ingress-controller.yaml’ saved [3129/3129]



In [231]:
!cat yaml/alb-ingress-controller.yaml

# Application Load Balancer (ALB) Ingress Controller Deployment Manifest.
# This manifest details sensible defaults for deploying an ALB Ingress Controller.
# GitHub: https://github.com/kubernetes-sigs/aws-alb-ingress-controller
apiVersion: apps/v1
kind: Deployment
metadata:
  labels:
    app.kubernetes.io/name: alb-ingress-controller
  name: alb-ingress-controller
  # Namespace the ALB Ingress Controller should run in. Does not impact which
  # namespaces it's able to resolve ingress resource for. For limiting ingress
  # namespace scope, see --watch-namespace.
  namespace: kube-system
spec:
  selector:
    matchLabels:
      app.kubernetes.io/name: alb-ingress-controller
  template:
    metadata:
      labels:
        app.kubernetes.io/name: alb-ingress-controller
    spec:
      containers:
        - name: alb-ingress-controller
          args:
            # Limit the namespace where this ALB Ingress Controller deployment will
            # resolve ingress resources. If left comment

alb-ingress-controller.yaml


-ingress-class=alb 行の後に、クラスター名、VPC ID、および AWS リージョン名を追加し、ファイルを保存して閉じます。


In [232]:
import re

before_str='- --ingress-class=alb'
after_str='''
            - --ingress-class=alb
            - --cluster-name={clusterName}
            - --aws-vpc-id={vpcID}
            - --aws-region=us-west-2
'''.format(clusterName=res_info["clusterName"],vpcID=res_info["vpcID"]).strip()

f = open('yaml/alb-ingress-controller.yaml','r')
body = f.read()
new_body = re.sub(before_str, after_str, body, flags=re.DOTALL)
f.close()

f = open('yaml/alb-ingress-controller.yaml','w')
f.write(new_body)
f.close()

In [233]:
!cat yaml/alb-ingress-controller.yaml


# Application Load Balancer (ALB) Ingress Controller Deployment Manifest.
# This manifest details sensible defaults for deploying an ALB Ingress Controller.
# GitHub: https://github.com/kubernetes-sigs/aws-alb-ingress-controller
apiVersion: apps/v1
kind: Deployment
metadata:
  labels:
    app.kubernetes.io/name: alb-ingress-controller
  name: alb-ingress-controller
  # Namespace the ALB Ingress Controller should run in. Does not impact which
  # namespaces it's able to resolve ingress resource for. For limiting ingress
  # namespace scope, see --watch-namespace.
  namespace: kube-system
spec:
  selector:
    matchLabels:
      app.kubernetes.io/name: alb-ingress-controller
  template:
    metadata:
      labels:
        app.kubernetes.io/name: alb-ingress-controller
    spec:
      containers:
        - name: alb-ingress-controller
          args:
            # Limit the namespace where this ALB Ingress Controller deployment will
            # resolve ingress resources. If left comment

##   persistentvolume等のデプロイ

In [234]:
!date

Fri Feb 14 11:31:30 UTC 2020


In [235]:
!kubectl apply -f yaml/class.yaml
!kubectl apply -f yaml/configmap.yaml

!kubectl apply -f yaml/rbac.yaml
!kubectl apply -f yaml/deployment.yaml

storageclass.storage.k8s.io/aws-efs created
configmap/efs-provisioner created
clusterrole.rbac.authorization.k8s.io/efs-provisioner-runner created
clusterrolebinding.rbac.authorization.k8s.io/run-efs-provisioner created
role.rbac.authorization.k8s.io/leader-locking-efs-provisioner created
rolebinding.rbac.authorization.k8s.io/leader-locking-efs-provisioner created
serviceaccount/efs-provisioner created
deployment.apps/efs-provisioner created


永続ボリュームクレーム

In [236]:
!kubectl apply -f yaml/claim.yaml

persistentvolumeclaim/fileserver-claim created


出来ていることを確認する

In [237]:
!kubectl get pod

NAME                               READY   STATUS    RESTARTS   AGE
efs-provisioner-79fcc97585-bwn6j   1/1     Running   0          3s


## wordpressのDeplyomentのデプロイ

In [238]:
!kubectl apply -f yaml/wordpress-deploy.yaml

deployment.apps/wordpress created


In [239]:
!kubectl get pod

NAME                               READY   STATUS    RESTARTS   AGE
efs-provisioner-79fcc97585-bwn6j   1/1     Running   0          5s
wordpress-6b7bdf676c-95tgk         0/1     Pending   0          1s
wordpress-6b7bdf676c-mh6pj         0/1     Pending   0          1s


## serviceのデプロイ

In [240]:
!kubectl apply -f yaml/wordpress-service.yaml

service/wordpress created


In [241]:
!kubectl get service

NAME         TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)        AGE
kubernetes   ClusterIP   172.20.0.1      <none>        443/TCP        179m
wordpress    NodePort    172.20.113.41   <none>        80:32002/TCP   1s


## ingressのデプロイ

In [242]:
!kubectl apply -f yaml/rbac-role.yaml
!kubectl apply -f yaml/alb-ingress-controller.yaml
!kubectl apply -f yaml/wordpress-ingress.yaml

clusterrole.rbac.authorization.k8s.io/alb-ingress-controller created
clusterrolebinding.rbac.authorization.k8s.io/alb-ingress-controller created
serviceaccount/alb-ingress-controller created
deployment.apps/alb-ingress-controller created
ingress.extensions/wordpress-ingress created


In [243]:
!kubectl get ingress

NAME                HOSTS   ADDRESS   PORTS   AGE
wordpress-ingress   *                 80      2s


In [244]:
!aws elbv2 describe-load-balancers

{
    "LoadBalancers": []
}


ステータスがActiveになるまで待ち、アドレスの取得


In [245]:
import json

while True:
    row = !aws elbv2 describe-load-balancers

    while row[0].find('{') == -1:
        del row[0]
    row[0] = "{"
    balancers =json.loads(''.join(row))
    
    if len(balancers["LoadBalancers"]) >= 1 :
        if balancers["LoadBalancers"][0]["State"]["Code"] == "active":
            address = balancers["LoadBalancers"][0]["DNSName"]
            print(address)
            break;
        print(balancers["LoadBalancers"][0]["State"]["Code"])
    
    print("waiting...")
    time.sleep(5)


waiting...
waiting...
provisioning
waiting...
provisioning
waiting...
provisioning
waiting...
provisioning
waiting...
provisioning
waiting...
provisioning
waiting...
provisioning
waiting...
provisioning
waiting...
provisioning
waiting...
provisioning
waiting...
provisioning
waiting...
a4c1c68a-default-wordpress-9484-2035555615.us-west-2.elb.amazonaws.com


httpアクセスを確認

In [246]:
import requests
import time

while True:
    
    try:
        r = requests.get('http://'+ address)
    except ConnectionError:
        print("not establish")

    if r.status_code == 200 :
        print("200 ok")
        break
        
    print("waiting...")
    time.sleep(10)


200 ok


In [247]:
!date

Fri Feb 14 11:34:14 UTC 2020


### クラスタ削除

In [248]:
# wp
!kubectl delete -f yaml/wordpress-deploy.yaml
!kubectl delete -f yaml/wordpress-service.yaml
!kubectl delete -f yaml/wordpress-ingress.yaml

# volume
!kubectl delete -f yaml/class.yaml
!kubectl delete -f yaml/configmap.yaml
!kubectl delete -f yaml/rbac.yaml
!kubectl delete -f yaml/deployment.yaml
!kubectl delete -f yaml/claim.yaml

# ingress
!kubectl delete -f yaml/rbac-role.yaml
!kubectl delete -f yaml/alb-ingress-controller.yaml

deployment.apps "wordpress" deleted
service "wordpress" deleted
ingress.extensions "wordpress-ingress" deleted
storageclass.storage.k8s.io "aws-efs" deleted
configmap "efs-provisioner" deleted
clusterrole.rbac.authorization.k8s.io "efs-provisioner-runner" deleted
clusterrolebinding.rbac.authorization.k8s.io "run-efs-provisioner" deleted
role.rbac.authorization.k8s.io "leader-locking-efs-provisioner" deleted
rolebinding.rbac.authorization.k8s.io "leader-locking-efs-provisioner" deleted
serviceaccount "efs-provisioner" deleted
deployment.apps "efs-provisioner" deleted
persistentvolumeclaim "fileserver-claim" deleted
clusterrole.rbac.authorization.k8s.io "alb-ingress-controller" deleted
clusterrolebinding.rbac.authorization.k8s.io "alb-ingress-controller" deleted
serviceaccount "alb-ingress-controller" deleted
deployment.apps "alb-ingress-controller" deleted


In [249]:
!kubectl get pod
!kubectl get ingress
!kubectl get service

NAME                               READY   STATUS        RESTARTS   AGE
efs-provisioner-79fcc97585-bwn6j   0/1     Terminating   0          2m53s
No resources found.
NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   172.20.0.1   <none>        443/TCP   3h2m
